In [89]:


! pip install chembl_webresource_client



In [90]:
import pandas as pd
from chembl_webresource_client.new_client import new_client

In [91]:
target = new_client.target
target_query = target.search('PPARgamma')
targets = pd.DataFrame.from_dict(target_query)
targets

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,[],Canis familiaris,Peroxisome proliferator-activated receptor gamma,13.0,False,CHEMBL1932903,"[{'accession': 'Q4U3Q4', 'component_descriptio...",SINGLE PROTEIN,9615
1,"[{'xref_id': 'P37231', 'xref_name': None, 'xre...",Homo sapiens,Peroxisome proliferator-activated receptor gamma,11.0,False,CHEMBL235,"[{'accession': 'P37231', 'component_descriptio...",SINGLE PROTEIN,9606
2,"[{'xref_id': 'P37238', 'xref_name': None, 'xre...",Mus musculus,Peroxisome proliferator-activated receptor gamma,11.0,False,CHEMBL2459,"[{'accession': 'P37238', 'component_descriptio...",SINGLE PROTEIN,10090
3,"[{'xref_id': 'O88275', 'xref_name': None, 'xre...",Rattus norvegicus,Peroxisome proliferator-activated receptor gamma,11.0,False,CHEMBL4797,"[{'accession': 'O88275', 'component_descriptio...",SINGLE PROTEIN,10116
4,"[{'xref_id': 'Q9UBK2', 'xref_name': None, 'xre...",Homo sapiens,Peroxisome proliferator-activated receptor gam...,9.0,False,CHEMBL6116,"[{'accession': 'Q9UBK2', 'component_descriptio...",SINGLE PROTEIN,9606
5,"[{'xref_id': 'Nuclear_receptor_coactivator_3',...",Homo sapiens,Nuclear receptor coactivator 3,9.0,False,CHEMBL1615382,"[{'accession': 'Q9Y6Q9', 'component_descriptio...",SINGLE PROTEIN,9606
6,[],Homo sapiens,PPAR alpha/gamma,8.0,False,CHEMBL2111325,"[{'accession': 'P37231', 'component_descriptio...",SELECTIVITY GROUP,9606
7,[],Homo sapiens,RXR alpha/PPAR gamma,8.0,False,CHEMBL2111394,"[{'accession': 'P37231', 'component_descriptio...",PROTEIN COMPLEX,9606
8,[],Homo sapiens,Peroxisome proliferator-activated receptor gam...,7.0,False,CHEMBL2095163,"[{'accession': 'P37231', 'component_descriptio...",PROTEIN-PROTEIN INTERACTION,9606
9,[],Homo sapiens,Peroxisome proliferator-activated receptor gam...,7.0,False,CHEMBL2096976,"[{'accession': 'P37231', 'component_descriptio...",PROTEIN-PROTEIN INTERACTION,9606


In [92]:
selected_target = targets.target_chembl_id[1]
selected_target

'CHEMBL235'

Selecting EC50 values reported in nM units

In [93]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="EC50", units="nM")

In [94]:
df = pd.DataFrame.from_dict(res)

In [95]:
df.head(3)

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,canonical_smiles,data_validity_comment,data_validity_description,document_chembl_id,document_journal,document_year,ligand_efficiency,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,potential_duplicate,qudt_units,record_id,relation,src_id,standard_flag,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,32233,[],CHEMBL658541,In vitro agonist efficacy against PPAR gamma a...,F,None,None,BAO_0000188,BAO_0000019,assay format,CC(/C=C/C(F)=C(/C)c1cc(C(C)(C)C)cc(C(C)(C)C)c1...,None,None,CHEMBL1136440,Bioorg. Med. Chem. Lett.,2003,None,CHEMBL111217,None,CHEMBL111217,7.20,False,http://www.openphacts.org/units/Nanomolar,251326,=,1,True,=,None,EC50,nM,None,63.0,CHEMBL235,Homo sapiens,Peroxisome proliferator-activated receptor gamma,9606,None,None,EC50,nM,UO_0000065,None,63.0
1,None,32234,[],CHEMBL656170,In vitro agonistic activity against PPAR gamma...,F,None,None,BAO_0000188,BAO_0000019,assay format,CC(/C=C/C(F)=C(/C)c1cc(C(C)(C)C)cc(C(C)(C)C)c1...,None,None,CHEMBL1136440,Bioorg. Med. Chem. Lett.,2003,None,CHEMBL111217,None,CHEMBL111217,8.19,False,http://www.openphacts.org/units/Nanomolar,251326,=,1,True,=,None,EC50,nM,None,6.4,CHEMBL235,Homo sapiens,Peroxisome proliferator-activated receptor gamma,9606,None,None,EC50,nM,UO_0000065,None,6.4
2,None,33386,[],CHEMBL658541,In vitro agonist efficacy against PPAR gamma a...,F,None,None,BAO_0000188,BAO_0000019,assay format,CCC(C)(C)c1cc(/C(C)=C(F)\C=C\C(C)=C\C(=O)O)c(O...,None,None,CHEMBL1136440,Bioorg. Med. Chem. Lett.,2003,None,CHEMBL130231,None,CHEMBL130231,7.29,False,http://www.openphacts.org/units/Nanomolar,251310,=,1,True,=,None,EC50,nM,None,51.0,CHEMBL235,Homo sapiens,Peroxisome proliferator-activated receptor gamma,9606,None,None,EC50,nM,UO_0000065,None,51.0


In [96]:
df.standard_type.unique()

array(['EC50'], dtype=object)

In [97]:
df.to_csv('PPARg_bioactivity_data_nM.csv', index=False)

Deleting missing data

In [98]:
df2 = df[df.value.notna()]
df2

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,canonical_smiles,data_validity_comment,data_validity_description,document_chembl_id,document_journal,document_year,ligand_efficiency,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,potential_duplicate,qudt_units,record_id,relation,src_id,standard_flag,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,32233,[],CHEMBL658541,In vitro agonist efficacy against PPAR gamma a...,F,None,None,BAO_0000188,BAO_0000019,assay format,CC(/C=C/C(F)=C(/C)c1cc(C(C)(C)C)cc(C(C)(C)C)c1...,None,None,CHEMBL1136440,Bioorg. Med. Chem. Lett.,2003,None,CHEMBL111217,None,CHEMBL111217,7.20,False,http://www.openphacts.org/units/Nanomolar,251326,=,1,True,=,None,EC50,nM,None,63.0,CHEMBL235,Homo sapiens,Peroxisome proliferator-activated receptor gamma,9606,None,None,EC50,nM,UO_0000065,None,63.0
1,None,32234,[],CHEMBL656170,In vitro agonistic activity against PPAR gamma...,F,None,None,BAO_0000188,BAO_0000019,assay format,CC(/C=C/C(F)=C(/C)c1cc(C(C)(C)C)cc(C(C)(C)C)c1...,None,None,CHEMBL1136440,Bioorg. Med. Chem. Lett.,2003,None,CHEMBL111217,None,CHEMBL111217,8.19,False,http://www.openphacts.org/units/Nanomolar,251326,=,1,True,=,None,EC50,nM,None,6.4,CHEMBL235,Homo sapiens,Peroxisome proliferator-activated receptor gamma,9606,None,None,EC50,nM,UO_0000065,None,6.4
2,None,33386,[],CHEMBL658541,In vitro agonist efficacy against PPAR gamma a...,F,None,None,BAO_0000188,BAO_0000019,assay format,CCC(C)(C)c1cc(/C(C)=C(F)\C=C\C(C)=C\C(=O)O)c(O...,None,None,CHEMBL1136440,Bioorg. Med. Chem. Lett.,2003,None,CHEMBL130231,None,CHEMBL130231,7.29,False,http://www.openphacts.org/units/Nanomolar,251310,=,1,True,=,None,EC50,nM,None,51.0,CHEMBL235,Homo sapiens,Peroxisome proliferator-activated receptor gamma,9606,None,None,EC50,nM,UO_0000065,None,51.0
3,None,33387,[],CHEMBL656170,In vitro agonistic activity against PPAR gamma...,F,None,None,BAO_0000188,BAO_0000019,assay format,CCC(C)(C)c1cc(/C(C)=C(F)\C=C\C(C)=C\C(=O)O)c(O...,None,None,CHEMBL1136440,Bioorg. Med. Chem. Lett.,2003,None,CHEMBL130231,None,CHEMBL130231,8.57,False,http://www.openphacts.org/units/Nanomolar,251310,=,1,True,=,None,EC50,nM,None,2.7,CHEMBL235,Homo sapiens,Peroxisome proliferator-activated receptor gamma,9606,None,None,EC50,nM,UO_0000065,None,2.7
4,None,35892,[],CHEMBL658541,In vitro agonist efficacy against PPAR gamma a...,F,None,None,BAO_0000188,BAO_0000019,assay format,C/C(=C/C=C/C(C)=C/C(=O)O)c1cc(-c2cc(F)cc(F)c2)...,None,None,CHEMBL1136440,Bioorg. Med. Chem. Lett.,2003,None,CHEMBL337393,None,CHEMBL337393,7.40,False,http://www.openphacts.org/units/Nanomolar,251312,=,1,True,=,None,EC50,nM,None,40.0,CHEMBL235,Homo sapiens,Peroxisome proliferator-activated receptor gamma,9606,None,None,EC50,nM,UO_0000065,None,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1379,None,19304533,[],CHEMBL4414384,Agonist activity at GAL4-tagged human PPAR rec...,B,None,None,BAO_0000188,BAO_0000357,single protein format,CN(CCOc1ccc(CC2SC(=O)NC2=O)cc1)c1ccccn1,None,None,CHEMBL4411340,J Med Chem,2019,"{'bei': '21.67', 'le': '0.42', 'lle': '5.25', ...",CHEMBL121,ROSIGLITAZONE,CHEMBL121,7.75,False,http://www.openphacts.org/units/Nanomolar,3223485,=,1,True,=,None,EC50,nM,None,18.0,CHEMBL235,Homo sapiens,Peroxisome proliferator-activated receptor gamma,9606,None,None,EC50,nM,UO_0000065,None,18.0
1380,None,20634352,[],CHEMBL4611395,Transactivation of GAL4-fused human PPARgamma ...,B,None,None,BAO_0000188,BAO_0000219,cell-based format,C[C@H](CCCOc1ccccc1Cn1c(C(F)(F)F)cnc1-c1ccc(C(...,None,None,CHEMBL4610040,Bioorg Med Chem 

Turning data in the "Value" column to float type

In [120]:
df2['value']=pd.to_numeric(df2['value'], downcast='float')

In [121]:
print(df2['value'].dtypes)

float32


Converting nM to uM

In [122]:
x=float(0.001)
df2['value']=x*df2['value']
df2['value']

0         0.063000
1         0.006400
2         0.051000
3         0.002700
4         0.040000
           ...    
1379      0.018000
1380    100.000008
1381    100.000008
1382    100.000008
1383      0.008870
Name: value, Length: 1384, dtype: float32

In [123]:
df2.to_csv('PPARg_bioactivity_data_no_missing_data_values .csv', index=False)

In [119]:
#df2 = pd.read_csv("PPARg_bioactivity_data_no_missing_data_value_changed_to_uM.csv")

#df2["New_Units"] = "uM"

#df.to_csv("PPARg_bioactivity_data_no_missing_data_value_changed_to_uM.csv", index=False)

Selecting EC50 values reported in uM units




In [104]:
activity = new_client.activity
res1 = activity.filter(target_chembl_id=selected_target).filter(standard_type="EC50", units="uM")

In [108]:
df3 = pd.DataFrame.from_dict(res1)


In [109]:
df4 = df3[df3.standard_value.notna()]

In [110]:
df4.to_csv('PPARg_bioactivity_data_uM.csv', index=False)

Merging the two CSV files in one

In [125]:
csv1 = pd.read_csv("PPARg_bioactivity_data_uM.csv")
csv2 = pd.read_csv("PPARg_bioactivity_data_no_missing_data_values .csv")
concate_data = pd.concat([csv1,csv2])
df5=concate_data
df5.to_csv('PPARg_bioactivity_data_all_uM.csv', index=False)
df5

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,canonical_smiles,data_validity_comment,data_validity_description,document_chembl_id,document_journal,document_year,ligand_efficiency,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,potential_duplicate,qudt_units,record_id,relation,src_id,standard_flag,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value,New_Units
0,NaN,105858,[],CHEMBL762741,Agonist activity on Gal4 chimeric human Peroxi...,F,NaN,NaN,BAO_0000188,BAO_0000219,cell-based format,CCC(Cc1ccc(OC)c(C(=O)NCc2ccc(OCCc3ccccc3)cc2)c...,NaN,NaN,CHEMBL1145047,J. Med. Chem.,2003.0,NaN,CHEMBL118483,NaN,CHEMBL118483,5.52,False,http://www.openphacts.org/units/Nanomolar,227404,=,1,True,=,NaN,EC50,nM,NaN,3000.00,CHEMBL235,Homo sapiens,Peroxisome proliferator-activated receptor gamma,9606,NaN,NaN,EC50,uM,UO_0000065,NaN,3.00000,NaN
1,NaN,108080,[],CHEMBL762741,Agonist activity on Gal4 chimeric human Peroxi...,F,NaN,NaN,BAO_0000188,BAO_0000219,cell-based format,CCCCC(Cc1ccc(OC)c(C(=O)NCc2ccc(C(F)(F)F)cc2)c1...,NaN,NaN,CHEMBL1145047,J. Med. Chem.,2003.0,NaN,CHEMBL331206,NaN,CHEMBL331206,5.60,False,http://www.openphacts.org/units/Nanomolar,227423,=,1,True,=,NaN,EC50,nM,NaN,2500.00,CHEMBL235,Homo sapiens,Peroxisome proliferator-activated receptor gamma,9606,NaN,NaN,EC50,uM,UO_0000065,NaN,2.50000,NaN
2,NaN,108763,[],CHEMBL760002,In vitro transactivation of human peroxisome p...,F,NaN,NaN,BAO_0000188,BAO_0000219,cell-based format,CCCc1cc(Oc2ccc(C(C)C)cc2)ccc1OCCCOc1cccc(C2SC(...,NaN,NaN,CHEMBL1136170,Bioorg. Med. Chem. Lett.,2003.0,NaN,CHEMBL94496,NaN,CHEMBL94496,6.78,False,http://www.openphacts.org/units/Nanomolar,174805,=,1,True,=,NaN,EC50,nM,NaN,167.00,CHEMBL235,Homo sapiens,Peroxisome proliferator-activated receptor gamma,9606,NaN,NaN,EC50,uM,UO_0000065,NaN,0.16700,NaN
3,NaN,108773,[],CHEMBL760002,In vitro transactivation of human peroxisome p...,F,NaN,NaN,BAO_0000188,BAO_0000219,cell-based format,CCCc1cc(Oc2ccc(Cl)cc2)ccc1OCCCOc1cccc(C2SC(=O)...,NaN,NaN,CHEMBL1136170,Bioorg. Med. Chem. Lett.,2003.0,NaN,CHEMBL420441,NaN,CHEMBL420441,6.78,False,http://www.openphacts.org/units/Nanomolar,174795,=,1,True,=,NaN,EC50,nM,NaN,165.00,CHEMBL235,Homo sapiens,Peroxisome proliferator-activated receptor gamma,9606,NaN,NaN,EC50,uM,UO_0000065,NaN,0.16500,NaN
4,NaN,108785,[],CHEMBL760002,In vitro transactivation of human peroxisome p...,F,NaN,NaN,BAO_0000188,BAO_0000219,cell-based format,CN(CCOc1ccc(CC2SC(=O)NC2=O)cc1)c1ccccn1,NaN,NaN,CHEMBL1136170,Bioorg. Med. Chem. Lett.,2003.0,NaN,CHEMBL121,ROSIGLITAZONE,CHEMBL121,7.70,True,http://www.openphacts.org/units/Nanomolar,174797,=,1,True,=,NaN,EC50,nM,NaN,20.00,CHEMBL235,Homo sapiens,Peroxisome proliferator-activated receptor gamma,9606,NaN,NaN,EC50,uM,UO_0000065,NaN,0.02000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1379,NaN,19304533,[],CHEMBL4414384,Agonist activity at GAL4-tagged human PPAR rec...,B,NaN,NaN,BAO_0000188,BAO_0000357,single protein format,CN(CCOc1ccc(CC2SC(=O)NC2=O)cc1)c1ccccn1,NaN,NaN,CHEMBL4411340,J Med Chem,2019.0,"{'bei': '21.67', 'le': '0.42', 'lle': '5.25', ...",CHEMBL121,ROSIGLITAZONE,CHEMBL121,7.75,False,http://www.openphacts.org/units/Nanomolar,3223485,=,1,True,=,NaN,EC50,nM,NaN,18.00,CHEMBL235,Homo sapiens,Peroxisome proliferator-activated receptor gamma,9606,NaN,NaN,EC50,nM,UO_0000065,NaN,0.01800,uM
1380,NaN,20634352,[],CHEMBL4611395,Transactivation of GAL4-fused human PPARgamma ...,B,NaN,NaN,BAO_0000188,BAO_0000219,cell-based format,C[C@H](CCCOc1ccccc1Cn1c(C(F)(F)F)cnc1-c1ccc(C(...,NaN,NaN,CHEMBL4610040,Bioorg Me

Data preprocessing

In [126]:
bioactivity_class = []
for i in df5.value:
  if float(i) >= 10:
    bioactivity_class.append("Partial Agonist")
  else:
    bioactivity_class.append("Full Agonist")


Iterate the molecule_chembl_id to a list

In [131]:
mol_cid = []
for i in df5.molecule_chembl_id:
  mol_cid.append(i)

Iterate canonical_smiles to a list

In [132]:
canonical_smiles = []
for i in df5.canonical_smiles:
  canonical_smiles.append(i)

Iterate standard_value to a list

In [133]:
value = []
for i in df5.value:
  value.append(i)


Combine the 4 lists into a dataframe

In [138]:
data_tuples = list(zip(mol_cid, canonical_smiles, bioactivity_class, value))
df6 = pd.DataFrame( data_tuples,  columns=['molecule_chembl_id', 'canonical_smiles', 'bioactivity_class', 'value_uM'])

In [139]:
df6

,molecule_chembl_id,canonical_smiles,bioactivity_class,value_uM
0,CHEMBL118483,CCC(Cc1ccc(OC)c(C(=O)NCc2ccc(OCCc3ccccc3)cc2)c...,Full Agonist,3.00000
1,CHEMBL331206,CCCCC(Cc1ccc(OC)c(C(=O)NCc2ccc(C(F)(F)F)cc2)c1...,Full Agonist,2.50000
2,CHEMBL94496,CCCc1cc(Oc2ccc(C(C)C)cc2)ccc1OCCCOc1cccc(C2SC(...,Full Agonist,0.16700
3,CHEMBL420441,CCCc1cc(Oc2ccc(Cl)cc2)ccc1OCCCOc1cccc(C2SC(=O)...,Full Agonist,0.16500
4,CHEMBL121,CN(CCOc1ccc(CC2SC(=O)NC2=O)cc1)c1ccccn1,Full Agonist,0.02000
...,...,...,...,...
3920,CHEMBL121,CN(CCOc1ccc(CC2SC(=O)NC2=O)cc1)c1ccccn1,Full Agonist,0.01800
3921,CHEMBL4648903,C[C@H](CCCOc1ccccc1Cn1c(C(F)(F)F)cnc1-c1ccc(C(...,Partial Agonist,100.00001
3922,CHEMBL4639259,C[C@H](CCCOc1ccccc1Cn1c(Cl)cnc1-c1ccc(C(F)(F)F...,Partial Agonist,100.00001
3923,CHEMBL4647502,C[C@H](CCCOc1ccccc1Cn1c(C(F)(F)F)cnc1-c1ccc(C#...,Partial Agonist,100.00001


In [141]:
df6.to_csv('PPARg_bioactivity_preprocessed_data.csv', index=False)